# 🏥 고양이 의료 문서 분석

이 노트북은 `data/은내과.pdf` 파일을 PaddleOCR로 분석하여 의료 데이터를 추출합니다.

## 주요 기능
- PDF → 이미지 변환
- PaddleOCR를 통한 테이블 데이터 추출
- 의료 검사 결과 구조화
- JSON 형태로 데이터 저장

## 1. 환경 설정 및 라이브러리 로드

In [1]:
# 필요한 라이브러리 설치 (첫 실행 시만)
!pip install PyMuPDF Pillow openai python-dotenv paddlepaddle paddleocr

In [2]:
import os
import base64
import json
from typing import List, Dict, Any
import fitz  # PyMuPDF
from PIL import Image
import io
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from IPython.display import display
import re
from datetime import datetime
from paddleocr import PaddleOCR

print("📦 라이브러리 로드 완료!")

📦 라이브러리 로드 완료!


## 2. PDF 처리 함수 정의

In [3]:
# def pdf_to_images(pdf_path: str, resolution_scale: float = 1) -> List[bytes]:
#     """PDF를 최적화된 해상도로 이미지 변환"""
#     doc = fitz.open(pdf_path)
#     images = []
    
#     print(f"📄 PDF 페이지 수: {len(doc)}")
#     print(f"⚙️ 해상도 스케일: {resolution_scale}x")
    
#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
        
#         # 해상도 조정 (기본값 1.0 = 원본 크기)
#         matrix = fitz.Matrix(resolution_scale, resolution_scale)
#         pix = page.get_pixmap(matrix=matrix)
        
#         # PNG 데이터 생성
#         img_data = pix.tobytes("png")
#         images.append(img_data)
        
#         # 크기 및 예상 처리시간 정보
#         size_mb = len(img_data) / (1024 * 1024)
#         estimated_time = size_mb * 2  # 대략적인 추정
        
#         print(f"  📑 페이지 {page_num + 1}: {len(img_data):,} bytes ({size_mb:.1f}MB)")
#         print(f"     ⏱️ 예상 PaddleOCR 처리시간: ~{estimated_time:.1f}초")
    
#     doc.close()
#     return images

def pdf_to_images(pdf_path: str, resolution_scale: float = 2.0) -> List[bytes]:
    """PDF를 고품질 이미지로 변환 (OCR 최적화)"""
    doc = fitz.open(pdf_path)
    images = []
    
    print(f"📄 PDF 페이지 수: {len(doc)}")
    print(f"⚙️ 해상도 스케일: {resolution_scale}x")
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        # 고품질 변환을 위한 매트릭스 설정
        mat = fitz.Matrix(resolution_scale, resolution_scale)
        
        # 고품질 픽스맵 생성 (anti-aliasing 활성화)
        pix = page.get_pixmap(
            matrix=mat,
            alpha=False,  # 투명도 제거로 용량 절약
            annots=True,  # 주석 포함
            clip=None     # 전체 페이지
        )
        
        # PNG 형식으로 변환 (압축률 조정)
        img_data = pix.tobytes("png")
        images.append(img_data)
        
        # 이미지 품질 정보 출력
        size_mb = len(img_data) / (1024 * 1024)
        print(f"  📑 페이지 {page_num + 1}:")
        print(f"     📐 픽셀 크기: {pix.width} x {pix.height}")
        print(f"     💾 파일 크기: {len(img_data):,} bytes ({size_mb:.1f}MB)")
        print(f"     📊 색상 공간: {pix.colorspace.name if pix.colorspace else 'Unknown'}")
        
        # 메모리 정리
        pix = None
    
    doc.close()
    return images

# 사용법
print("🎛️ 해상도 설정 선택:")
print("1️⃣ 빠른 처리: resolution_scale=0.8")
print("2️⃣ 균형잡힌 품질(원본 크기): resolution_scale=1.0") 
print("3️⃣ 고품질: resolution_scale=1.5")

def image_to_base64(image_data: bytes) -> str:
    """이미지를 base64로 인코딩"""
    return base64.b64encode(image_data).decode('utf-8')

def save_image_preview(image_data: bytes, filename: str):
    """이미지 미리보기 저장"""
    image = Image.open(io.BytesIO(image_data))
    # 미리보기용 크기 조정
    image.thumbnail((800, 800))
    image.save(filename)
    print(f"🖼️ 미리보기 저장: {filename}")

print("🔧 PDF 처리 함수 정의 완료!")

🎛️ 해상도 설정 선택:
1️⃣ 빠른 처리: resolution_scale=0.8
2️⃣ 균형잡힌 품질(원본 크기): resolution_scale=1.0
3️⃣ 고품질: resolution_scale=1.5
🔧 PDF 처리 함수 정의 완료!


## 3. PDF 이미지 변환 및 미리보기

In [8]:
print("📂 파일 선택 (PDF 또는 이미지)")
print("=" * 40)

# 사용자 입력 받기 (data/ 폴더 기본 경로)
filename = input("📄 data/ 폴더 내 파일명을 입력하세요 (예: 은내과.PDF, 검사결과.png): ").strip()

# 빈 입력 시 기본값 사용
if not filename:
    filename = "20241107.jpg"
    print(f"🔹 기본 파일 사용: {filename}")

# data/ 경로와 결합
file_path = f"../data/{filename}"
print(f"📁 파일 경로: {file_path}")

# 파일 확장자 확인
file_ext = filename.lower().split('.')[-1]
is_pdf = file_ext == 'pdf'
is_image = file_ext in ['png', 'jpg', 'jpeg', 'tiff', 'bmp']

print(f"📋 파일 형식: {file_ext.upper()}")

# output 폴더 생성
os.makedirs("output", exist_ok=True)

if is_pdf:
    # PDF 처리
    print("🔄 PDF → 이미지 변환 시작...")
    pdf_images = pdf_to_images(file_path, resolution_scale=2.0)
    
    # 미리보기 이미지 저장
    for i, img_data in enumerate(pdf_images):
        save_image_preview(img_data, f"output/page_{i+1}_preview.png")
    
    print(f"✅ PDF 변환 완료! {len(pdf_images)}개 페이지 처리됨")

elif is_image:
    # 이미지 파일 처리
    print("🖼️ 이미지 파일 로드 중...")
    
    with open(file_path, 'rb') as f:
        img_data = f.read()
    
    # 단일 이미지를 리스트에 담기
    pdf_images = [img_data]
    
    # 미리보기 이미지 저장
    save_image_preview(img_data, "output/image_preview.png")
    
    print(f"✅ 이미지 로드 완료! 1개 이미지 처리됨")

else:
    print(f"❌ 지원하지 않는 파일 형식입니다: {file_ext}")
    print("📋 지원 형식: PDF, PNG, JPG, JPEG, TIFF, BMP")
    pdf_images = []

if pdf_images:
    print(f"🎯 총 {len(pdf_images)}개 이미지가 분석 준비 완료!")
    print("📋 다음 섹션(PaddleOCR 분석)을 실행할 수 있습니다!")
else:
    print("⚠️ 분석할 이미지가 없습니다. 올바른 파일을 선택해주세요.")

📂 파일 선택 (PDF 또는 이미지)
🔹 기본 파일 사용: 20241107.jpg
📁 파일 경로: ../data/20241107.jpg
📋 파일 형식: JPG
🖼️ 이미지 파일 로드 중...
🖼️ 미리보기 저장: output/image_preview.png
✅ 이미지 로드 완료! 1개 이미지 처리됨
🎯 총 1개 이미지가 분석 준비 완료!
📋 다음 섹션(PaddleOCR 분석)을 실행할 수 있습니다!


## 4. PaddleOCR을 이용한 데이터 추출

In [14]:
def extract_medical_data_with_paddleocr(image_data: bytes, page_num: int = 0) -> pd.DataFrame:
    """PaddleOCR로 혈액검사 데이터 추출 및 DataFrame 반환"""
    
    print(f"🔍 PaddleOCR로 페이지 {page_num + 1} 분석 중...")
    
    # PaddleOCR 초기화 (영어)
    ocr = PaddleOCR(use_angle_cls=True, lang='en')

    try:
        # 디버깅: 이미지 크기 확인
        print(f"📊 이미지 데이터 크기: {len(image_data):,} bytes")
        
        # PIL Image로 변환하여 이미지 정보 확인
        image = Image.open(io.BytesIO(image_data))
        print(f"🖼️ 이미지 크기: {image.size}, 모드: {image.mode}")
        
        # OCR 실행 - PIL Image 객체 사용
        result = ocr.ocr(image_data)
        
        # 결과 구조 디버깅
        print(f"🔍 OCR 결과 타입: {type(result)}")
        print(f"🔍 OCR 결과 길이: {len(result) if result else 0}")
        
        if result and len(result) > 0:
            print(f"🔍 첫 번째 결과: {type(result[0])}, 길이: {len(result[0]) if result[0] else 0}")
        
        # 결과 확인 및 처리
        if not result or not result[0]:
            print("❌ OCR 결과가 없습니다.")
            print("💡 가능한 원인:")
            print("   - 이미지 품질이 낮음")
            print("   - 텍스트가 너무 작음")
            print("   - 이미지가 회전됨")
            print("   - 언어 설정 문제")
            
            # 한국어 모델로 재시도
            print("🔄 한국어 모델로 재시도...")
            ocr_kr = PaddleOCR(use_angle_cls=True, lang='korean')
            result = ocr_kr.ocr(image_data)
            
            if not result or not result[0]:
                print("❌ 한국어 모델로도 인식 실패")
                empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
                empty_df.attrs['exam_date'] = "Not Found"
                return empty_df
            else:
                print("✅ 한국어 모델로 텍스트 인식 성공!")
        
        # 텍스트 추출
        extracted_texts = []
        for line in result[0]:
            if len(line) >= 2 and len(line[1]) >= 2:
                text = line[1][0]
                confidence = line[1][1]
                print(f"📝 텍스트: '{text}' (신뢰도: {confidence:.2f})")
                if confidence > 0.3:  # 신뢰도 임계값 낮춤
                    extracted_texts.append(text.strip())
        
        print(f"📝 추출된 텍스트 수: {len(extracted_texts)}")
        
        if not extracted_texts:
            print("⚠️ 신뢰도 기준을 만족하는 텍스트가 없습니다.")
            # 신뢰도 무시하고 모든 텍스트 추출
            for line in result[0]:
                if len(line) >= 2:
                    text = line[1][0]
                    extracted_texts.append(text.strip())
            print(f"📝 신뢰도 무시한 텍스트 수: {len(extracted_texts)}")
        
        # 디버깅용: 추출된 텍스트 모두 출력
        print("🔍 추출된 모든 텍스트:")
        for i, text in enumerate(extracted_texts):
            print(f"  {i+1}: {text}")
        
        # 검사일시 추출
        exam_date = extract_exam_date(extracted_texts)
        print(f"📅 검사일시: {exam_date}")
        
        # 혈액검사 데이터 추출
        blood_tests = extract_blood_test_data(extracted_texts)
        
        # DataFrame 생성
        if blood_tests:
            df = pd.DataFrame(blood_tests)
            df.attrs['exam_date'] = exam_date
            
            print("✅ PaddleOCR로 추출된 혈액검사 데이터:")
            display(df)
            print(f"\n📈 총 {len(df)}개 혈액검사 항목 추출됨")
            
            return df
        else:
            print("⚠️ 혈액검사 데이터를 찾을 수 없습니다.")
            empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
            empty_df.attrs['exam_date'] = exam_date
            return empty_df
            
    except Exception as e:
        print(f"❌ PaddleOCR 분석 오류: {e}")
        import traceback
        print(f"🔍 상세 오류: {traceback.format_exc()}")
        empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
        empty_df.attrs['exam_date'] = "Not Found"
        return empty_df

print("🔧 PaddleOCR 데이터 추출 함수 정의 완료!")

print("🔧 PaddleOCR 데이터 추출 함수 정의 완료!")


def extract_exam_date(texts: List[str]) -> str:
    """텍스트에서 검사일시 추출 (다양한 날짜 형식 지원)"""
    
    # 날짜 패턴 정의 (우선순위 순)
    date_patterns = [
        r'\b(20\d{2}[-:/]\d{1,2}[-:/]\d{1,2})\b',  # yyyy-mm-dd, yyyy/mm/dd, yyyy:mm:dd
        r'\b(\d{2}[-:/]\d{1,2}[-:/]\d{1,2})\b',    # yy-mm-dd, yy/mm/dd, yy:mm:dd
        r'\b(20\d{2}\.\d{1,2}\.\d{1,2})\b',       # yyyy.mm.dd
        r'\b(\d{2}\.\d{1,2}\.\d{1,2})\b',         # yy.mm.dd
        r'\b(20\d{2}\s+\d{1,2}\s+\d{1,2})\b',     # yyyy mm dd (공백)
        r'\b(\d{2}\s+\d{1,2}\s+\d{1,2})\b'        # yy mm dd (공백)
    ]
    
    found_dates = []
    
    for text in texts:
        # 생년월일 관련 키워드가 포함된 텍스트는 제외
        birth_keywords = ['생일', '생년월일', 'Birth', 'DOB', 'Born', '출생']
        if any(keyword in text for keyword in birth_keywords):
            continue
            
        # 검사일시 관련 키워드 우선 검색
        exam_keywords = ['검사일시', '검사날짜', 'Date', 'Time', '일시']
        is_exam_date_line = any(keyword in text for keyword in exam_keywords)
        
        for pattern in date_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                try:
                    # 날짜 형식 정규화
                    normalized_date = normalize_date(match)
                    if normalized_date:
                        # 검사일시 라벨이 있는 경우 우선순위 부여
                        priority = 1 if is_exam_date_line else 2
                        found_dates.append((normalized_date, priority, match))
                except:
                    continue
    
    if not found_dates:
        return "Not Found"
    
    # 우선순위로 정렬 (검사일시 라벨이 있는 것 우선, 그 다음 최신 날짜)
    found_dates.sort(key=lambda x: (x[1], -datetime.strptime(x[0], '%Y-%m-%d').timestamp()))
    
    return found_dates[0][0]

def normalize_date(date_str: str) -> str:
    """다양한 날짜 형식을 yyyy-mm-dd로 정규화"""
    
    # 구분자 통일 (-, /, :, ., 공백을 모두 -로)
    normalized = re.sub(r'[-:/.\s]+', '-', date_str.strip())
    
    try:
        parts = normalized.split('-')
        if len(parts) != 3:
            return None
            
        year, month, day = parts
        
        # 2자리 연도를 4자리로 변환
        if len(year) == 2:
            year_int = int(year)
            if year_int <= 30:  # 00-30은 2000년대
                year = f"20{year}"
            else:  # 31-99는 1900년대
                year = f"19{year}"
        
        # 월, 일을 2자리로 패딩
        month = month.zfill(2)
        day = day.zfill(2)
        
        # 유효한 날짜인지 검증
        datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')
        
        return f"{year}-{month}-{day}"
        
    except (ValueError, IndexError):
        return None

def extract_blood_test_data(texts: List[str]) -> List[Dict[str, str]]:
    """텍스트에서 혈액검사 데이터 추출"""
    
    # 혈액검사 항목 패턴 (대소문자 구분 없음)
    blood_test_patterns = {
        'RBC': r'\b(RBC|적혈구)\b',
        'WBC': r'\b(WBC|백혈구)\b',
        'HGB': r'\b(HGB|HB|헤모글로빈|혈색소)\b',
        'HCT': r'\b(HCT|헤마토크리트)\b',
        'PLT': r'\b(PLT|혈소판)\b',
        'MCV': r'\b(MCV)\b',
        'MCH': r'\b(MCH)\b',
        'MCHC': r'\b(MCHC)\b',
        'GLU': r'\b(GLU|포도당|혈당)\b',
        'CREA': r'\b(CREA|크레아티닌)\b',
        'BUN': r'\b(BUN|요소질소)\b',
        'ALT': r'\b(ALT|GPT)\b',
        'AST': r'\b(AST|GOT)\b',
        'ALP': r'\b(ALP|알카리포스파타제)\b',
        'TP': r'\b(TP|총단백)\b',
        'ALB': r'\b(ALB|알부민)\b'
    }
    
    # 단위 패턴
    unit_patterns = [
        r'\b(g/dL|mg/dL|×10³/μL|×10⁶/μL|/μL|fl|pg|%|U/L|IU/L)\b',
        r'\b(g/dl|mg/dl|x10³/μl|x10⁶/μl|/μl)\b',
        r'\b(mil/cmm|thou/cmm)\b'
    ]
    
    blood_tests = []
    
    # 각 텍스트 라인 분석
    for text in texts:
        text_upper = text.upper()
        
        # 혈액검사 항목 찾기
        for test_name, pattern in blood_test_patterns.items():
            if re.search(pattern, text_upper):
                # 숫자 값 추출 (소수점 포함)
                number_match = re.search(r'\b(\d+\.?\d*)\b', text)
                if number_match:
                    result = number_match.group(1)
                    
                    # 단위 추출
                    unit = ""
                    for unit_pattern in unit_patterns:
                        unit_match = re.search(unit_pattern, text)
                        if unit_match:
                            unit = unit_match.group(1)
                            break
                    
                    blood_tests.append({
                        "혈액항목": test_name,
                        "측정값": result,
                        "단위": unit
                    })
                    break
    
    # 중복 제거 (같은 항목이 여러 번 추출된 경우)
    seen_tests = set()
    unique_tests = []
    for test in blood_tests:
        test_key = test["혈액항목"]
        if test_key not in seen_tests:
            seen_tests.add(test_key)
            unique_tests.append(test)
    
    return unique_tests

print("🔧 PaddleOCR 데이터 추출 함수 정의 완료!")

🔧 PaddleOCR 데이터 추출 함수 정의 완료!
🔧 PaddleOCR 데이터 추출 함수 정의 완료!
🔧 PaddleOCR 데이터 추출 함수 정의 완료!


## 5. 의료 문서 분석 실행

In [15]:
print("🔍 PaddleOCR 의료 문서 분석 시작...")
print("=" * 50)

for i, img_data in enumerate(pdf_images):
    print(f"\n📑 페이지 {i + 1} 분석 중...")
    analysis_result = extract_medical_data_with_paddleocr(img_data, i)


/tmp/ipykernel_110573/1964690925.py:7: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang='korean')
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.


🔍 PaddleOCR 의료 문서 분석 시작...

📑 페이지 1 분석 중...
🔍 PaddleOCR로 페이지 1 분석 중...


Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/korean_PP-OCRv5_mobile_rec`.
/tmp/ipykernel_110573/1964690925.py:18: DeprecationWarning: Please use `predict` instead.
  result = ocr.ocr(image_data)
Not supported input data type! Only `numpy.ndarray` and

📊 이미지 데이터 크기: 213,793 bytes
🖼️ 이미지 크기: (794, 1123), 모드: RGB
🔍 OCR 결과 타입: <class 'list'>
🔍 OCR 결과 길이: 0
❌ OCR 결과가 없습니다.
💡 가능한 원인:
   - 이미지 품질이 낮음
   - 텍스트가 너무 작음
   - 이미지가 회전됨
   - 언어 설정 문제
🔄 한국어 모델로 재시도...


Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/korean_PP-OCRv5_mobile_rec`.
/tmp/ipykernel_110573/1964690925.py:39: DeprecationWarning: Please use `predict` instead.
  result = ocr_kr.ocr(image_data)
Not supported input data type! Only `numpy.ndarray` 

❌ 한국어 모델로도 인식 실패
